# Esercitazione 1 di Tecnologie del Linguaggio Naturale - Utilizzo di risorse lessicografiche per la concept similarity e la WSD

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

## Conceptual similarity with WordNet

Consegna: dati in input due termini, il task di conceptual similarity consiste nel fornire un punteggio numerico di similarità che ne indichi la vicinanza semantica.

In [58]:
!curl "https://raw.githubusercontent.com/msavva/transphoner/master/data/wordsim353.csv" -o WordSim353.csv

import nltk
nltk.download('wordnet')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7079  100  7079    0     0  65906      0 --:--:-- --:--:-- --:--:-- 66783


[nltk_data] Downloading package wordnet to /home/lorenzo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Si va a parsare il dataset: questo è composto da 353 $3$-ple della forma $(termine1, \: termine2, \: media)$.

In [59]:
import csv
from typing import List

def load_dataset(path) -> List[tuple[str, str, float]]:
  dataset = []
  with open(path, newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        dataset.append((row['Word 1'], row['Word 2'], row['Human (mean)']))
  return dataset

dataset = load_dataset('WordSim353.csv')
dataset[:3]

[('love', 'sex', '6.77'),
 ('tiger', 'cat', '7.35'),
 ('tiger', 'tiger', '10.00')]

Il calcolo della profondità del senso attuale viene implementato come una **ricerca in ampiezza** sulla **multi-gerarchia**. È necessario implementare una (variante di una) strategia di ricerca in quanto possono esistere vari percorsi che dal dato senso portano a una radice della multi-gerarchia, alcuni con lunghezza inferiori ad altri. La profondità di un senso, in definitiva, è la lunghezza del cammino minimo che porta dal dato senso a una radice (la più vicina) della multi-gerarchia. È stata inoltre implementata la profondità massima di un senso, necessaria per il calcolo di metriche di similarità.

In [60]:
from nltk.corpus import wordnet as wn

# Compute the (min) depth of a given sense (the distance between the sense and the closest hierarchy root)
def sense_depth(sense) -> int:
    depth = 0
    stack = [sense]

    # Breadth-First Search over hypernyms
    while True:
        new_stack = []
        for node in stack:
            hypernyms = node.hypernyms()
            if not hypernyms: # the current node is a hierarchy root
                return depth
            new_stack.extend(node.hypernyms())

        stack = new_stack # On the next iteration, the new hypernyms will be evaluated (expanded)
        depth += 1

# Compute the maximum depth of a given sense (the distance between the sense and the most distant hierarchy root)
def max_sense_depth(sense) -> int:
  depth = -1
  stack = [sense]

  # Breadth-First Search over hypernyms
  while stack:
      new_stack = []
      for node in stack:
          hypernyms = node.hypernyms()
          if hypernyms:
            new_stack.extend(node.hypernyms())

      stack = new_stack # On the next iteration, the new hypernyms will be evaluated (expanded)
      depth += 1

  return depth

In [61]:
# Check if the computed depths match the ones computed by NTLK
syn = wn.synsets(dataset[0][0])[0]
if sense_depth(syn) != syn.min_depth(): print(False)
if max_sense_depth(syn) != syn.max_depth(): print(False)

# Extensive check
for record in dataset:
  for term1 in record[0]:
    for sense in wn.synsets(term1):
      if sense_depth(sense) != sense.min_depth(): print(False)
      if max_sense_depth(sense) != sense.max_depth(): print(False)
  for term2 in record[1]:
    for sense in wn.synsets(term2):
      if sense_depth(sense) != sense.min_depth(): print(False)
      if max_sense_depth(sense) != sense.max_depth(): print(False)

### Lowest Common Subsumer

Il *lowest common subsumer* (LCS) è l'iperonimo comune tra i due sensi che si trova più in profondità all'interno della multi-gerarchia (tra i possibili iperonimi in comune è quindi quello più vicino ai due sensi, o più succintamente l'*antenato comune più specifico*). In maniera consistente con la libreria NLTK, il LCS tra due sensi di POS di natura differente (e.g. un senso relativo a un verbo e uno relativo a un sostantivo) non esiste.

In [62]:
# Compute the lowest common subsumer between two senses
def lowest_common_subsumer(sense1, sense2):

    # The LCS of two senses referring to different POS doesn't exist (performance improvement, the same resulted will be computed further)
    if sense1.pos() != sense2.pos():
        return None

    sense1_expansion = [sense1]
    sense2_expansion = [sense2]

    # Get the list of hypernyms of the first sense
    for node in sense1_expansion:
      current_hypernym = node.hypernyms()
      if current_hypernym: sense1_expansion.extend(current_hypernym)
    # Get the list of hypernyms of the second sense
    for node in sense2_expansion:
      current_hypernym = node.hypernyms()
      if current_hypernym: sense2_expansion.extend(current_hypernym)

    common_subsumers = set([sense for sense in sense1_expansion if sense in sense2_expansion])

    # Compute the lowest common subsumer
    lcs = None
    lcs_depth = 0

    for cs in common_subsumers:
      ch_depth = max_sense_depth(cs)
      if ch_depth >= lcs_depth:
        lcs = cs
        lcs_depth = ch_depth

    return lcs

In [63]:
# Check if the computed LCS match the one computed by NTLK
s1 = wn.synsets(dataset[1][0])
s2 = wn.synsets(dataset[1][1])
for sense1 in s1:
  for sense2 in s2:
    if lowest_common_subsumer(sense1, sense2) and not(lowest_common_subsumer(sense1, sense2) in sense1.lowest_common_hypernyms(sense2)):
      print(False)

# Extensive check for LCS
for record in dataset:
  s1 = wn.synsets(record[0])
  s2 = wn.synsets(record[1])
  for sense1 in s1:
    for sense2 in s2:
      if lowest_common_subsumer(sense1, sense2) and not(lowest_common_subsumer(sense1, sense2) in sense1.lowest_common_hypernyms(sense2)):
        print(False)

Per il calcolo della distanza tra due sensi (utilizzata in alcune metriche di similarità) è necessario distinguere se il LCS tra i due sensi è presente o meno: in caso sia presente, il calcolo della distanza è immediato; in caso non sia presente, si è deciso (*assunzione di lavoro*) di sommare le profondità massime dei due sensi e aggiungere uno.

In [64]:
def length_between_senses(sense1, sense2) -> int:
  lcs = lowest_common_subsumer(sense1, sense2)
  if lcs:
      lcs_depth = sense_depth(lcs)
      return (max_sense_depth(sense1) - lcs_depth) + (max_sense_depth(sense2) - lcs_depth)
  else:
      return max_sense_depth(sense1) + max_sense_depth(sense2) + 1

### Metriche di similarità

Si implementano delle metriche di similarità tra sensi:

- Wu & Palmer: $cs(s_1, s_2) = \frac{2 \cdot depth(LCS)}{depth(s_1) + depth(s_2)}$
- Shortest Path: $sim_{path}(s_1, s_2) = 2 \cdot depthMax - len(s_1, s_2)$
- Leakcock & Chodorow: $sim_{ln}(s_1, s_2) = - \log \frac{len(s_1, s_2)}{2 \cdot depthMax}$

È importante notare che le metriche lavorano su sensi, e non su termini.

La costante $depthMax$ è pari a [19 per i termini, 12 per i verbi, 0 per aggettivi e avverbi](https://github.com/nltk/wordnet/blob/ce91915ae38a341ae845be4d825ef6003cddf395/wn/constants.py#L31C1-L31C1). Invece di ricalcolarla a ogni chiamata (come invece fa NLTK nella sua attuale versione), si decide invece di mantenerla *hard-coded* nel codice (come avviene nella versione candidata di NLTK).

In [65]:
import math

def get_depth_max(sense1, sense2) -> int:
    depth_max_s1 = 0
    depth_max_s2 = 0

    match sense1.pos():
        case 'n': depth_max_s1 = 19
        case 'v': depth_max_s1 = 12
    match sense2.pos():
        case 'n': depth_max_s2 = 19
        case 'v': depth_max_s2 = 12

    return depth_max_s1 if depth_max_s1 > depth_max_s2 else depth_max_s2

def wup_similarity(sense1, sense2) -> float:
    lcs = lowest_common_subsumer(sense1, sense2)
    if lcs: return (2 * max_sense_depth(lcs)) / (max_sense_depth(sense1) + max_sense_depth(sense2))
    else: return 0

def path_similarity(sense1, sense2) -> int:
    depth_max = get_depth_max(sense1, sense2)
    return (2 * depth_max) - length_between_senses(sense1, sense2)

def lch_similarity(sense1, sense2) -> float:
    depth_max = get_depth_max(sense1, sense2)
    depth_max = 1 if depth_max == 0 else depth_max # Needed to avoid DivisionByZero
    return - math.log((length_between_senses(sense1, sense2) + 1) / (2 * depth_max + 1))

In [66]:
# Check how the computed similarity metrics perform compared to the ones computed by NTLK
s1 = wn.synsets(dataset[1][0])[0]
s2 = wn.synsets(dataset[1][1])[0]

print("WUP:")
print(" ", wup_similarity(s1, s2))
print(" ", s1.wup_similarity(s2))

print("Path:")
print(" ", path_similarity(s1, s2))
print(" ", s1.path_similarity(s2))

print("LCH:")
print(" ", lch_similarity(s1, s2))
print(" ", s1.lch_similarity(s2))

WUP:
  0.5
  0.5454545454545454
Path:
  28
  0.09090909090909091
LCH:
  1.2656663733312759
  1.2396908869280152


In relazione al risultato della *path similarity*, la libreria NLTK calcola quest'ultima come $\frac{1}{len(s1, s2) + 1}$. Questa non è però la definizione di PS vista a lezione, a cui si è invece deciso di far riferimento. Analogamente, nella *lch similarity*, viene aggiunto $1$ solo al dividendo e non anche al divisore, differendo da quanto visto a lezione.

In generale, piuttosto che ricercare una *result-parity* con l'implementazione delle metriche di NLTK, si è deciso di seguire quanto visto a lezione, fornendo un'implementazione minimale delle metriche che cercasse allo stesso tempo di coglierne l'essenza.

### Similarità massima e correlazione

Data una coppia di termini, si va a calcolare la coppia di sensi dei due termini che massimizza la loro similarità. Si esegue questo calcolo per ogni coppia presente nel dataset, per poi andare a calcolare la correlazione rispetto ai valori di similarità medi forniti da soggetti umani.

In [67]:
import numpy as np

def compute_max_similarity(term1, term2, similarity = wup_similarity):

    max_similarity = 0

    senses1 = wn.synsets(term1)
    senses2 = wn.synsets(term2)

    for sn1 in senses1:
        for sn2 in senses2:
            current_similarity = similarity(sn1, sn2)
            max_similarity = current_similarity if current_similarity > max_similarity else max_similarity

    return max_similarity

In [68]:
computed_similarities = np.array([])
dataset_similarities = np.array([])

similarity_function = wup_similarity # Set here the preferred similarity metrics {wup_similarity, lch_similarity, path_similarity}

for record in dataset:
    dataset_similarities = np.append(dataset_similarities, record[2])

    ms = compute_max_similarity(record[0], record[1], similarity_function)
    computed_similarities = np.append(computed_similarities, ms)

print(computed_similarities[:10])
dataset_similarities = dataset_similarities.astype(float) # From Unicode to float
print(dataset_similarities[:10])

[0.90909091 0.96296296 1.         0.85714286 0.8        0.58823529
 0.7        0.70588235 0.         0.9       ]
[ 6.77  7.35 10.    7.46  7.62  7.58  5.77  6.31  7.5   6.77]


Sui valori di similarità ottenuti si va a calcolare la correlazione rispetto ai valori contenuti nel dataset.

Correlazione prevista: 30-35%.

In [69]:
from scipy import stats

print("Correlations computed on", similarity_function.__name__.replace("_", " "), "")
res_pear = stats.pearsonr(dataset_similarities, computed_similarities)
print("  Pearson:", res_pear.statistic)
res_spear = stats.spearmanr(dataset_similarities, computed_similarities)
print("  Spearman:", res_spear.statistic)

Correlations computed on wup similarity 
  Pearson: 0.27563185573720034
  Spearman: 0.32205979107230365


Per valutare la bontà della nostra implementazione si va quindi a calcolare la correlazione sulle similarità massime ottenute tramite la libreria NLTK.

In [70]:
def compute_max_similarity_with_ntlk(term1, term2, similarity):

    max_similarity = 0

    senses1 = wn.synsets(term1)
    senses2 = wn.synsets(term2)

    for sn1 in senses1:
        for sn2 in senses2:

            match similarity.__name__:
              case "path_similarity":
                current_similarity = sn1.path_similarity(sn2)
              case "lch_similarity":
                current_similarity = sn1.lch_similarity(sn2) if sn1.pos() == sn2.pos() else 0
              case _:
                current_similarity = sn1.wup_similarity(sn2)

            max_similarity = current_similarity if current_similarity > max_similarity else max_similarity

    return max_similarity

In [71]:
nltk_similarities = np.array([])

for record in dataset:
    ms = compute_max_similarity_with_ntlk(record[0], record[1], similarity_function)
    nltk_similarities = np.append(nltk_similarities, ms)

print("Correlations computed on", similarity_function.__name__.replace("_", " "), "")

res_lib_pear = stats.pearsonr(dataset_similarities, nltk_similarities)
print("  Pearson:", res_lib_pear.statistic)
res_lib_spear = stats.spearmanr(dataset_similarities, nltk_similarities)
print("  Spearman:", res_lib_spear.statistic)

print("\n  Difference (Pearson):", res_lib_pear.statistic - res_pear.statistic)
print("  Difference (Spearman):", res_lib_spear.statistic - res_spear.statistic)

Correlations computed on wup similarity 
  Pearson: 0.297440832551365
  Spearman: 0.33850273836806116

  Difference (Pearson): 0.02180897681416466
  Difference (Spearman): 0.016442947295757515


## Word Sense Disambiguation

Consegna: implementare l'algoritmo Lesk.

  1. Estrarre 50 frasi dal corpus SemCor e disambiguare (almeno) un sostantivo per frase. Calcolare l'accuratezza del sistema implementato sulla base dei sensi annotati in SemCor.
  2. Randomizzare la selezione delle 50 frasi e la selezione del termine da disambiguare, e restituire l'accuratezza media su (per esempio) 10 esecuzioni del programma.
  - Opzionale: implementare l'algoritmo Corpus Lesk utilizzando Sem-Cor.

### Algoritmo di Lesk

La seguente implementazione segue in maniera fedele quella vista a lezione.

In [72]:
def simplified_lesk(word, sentence):

  word_senses = wn.synsets(word)
  best_sense = word_senses[0]

  if len(word_senses) == 1: return best_sense

  max_overlap = 0

  # The context is the corresponding set of words contained in the given sentence (minus non-alphanumerical chars)
  context = set(remove_non_alphanumerical_chars(sentence).split(' '))

  for word_sense in word_senses:

    # Compute current sense signature (definition + examples)
    signature = set(remove_non_alphanumerical_chars(word_sense.definition()).split(' '))
    for example in word_sense.examples():
      # Add current example words to the signature (minus non-alphanumerical chars)
      signature.update(remove_non_alphanumerical_chars(example).split(' '))

    overlap = len(signature.intersection(context)) # Compute the overlap between current signature and the given context

    if overlap > max_overlap:
      max_overlap = overlap
      best_sense = word_sense

  return best_sense

def remove_non_alphanumerical_chars(s):
  return ''.join(filter(lambda c: c.isalnum() or c.isspace(), s))

Si esegue l'algoritmo di Lesk sulla frase d'esempio *the bank can guarantee deposits will eventually cover future tuition costs because it invests in adjustable-rate mortgage securities*, dove si vuole disambiguare il termine *bank*.

Per un parlante Inglese umano è evidente come *bank* in questa frase si riferisca a un'istituzione finanziaria (corrispondente al synset `'depository_financial_institution.n.01'`) e non ad altri possibili sensi, come, per esempio, quello di *banchina* (`'bank.n.01'`) o di insieme di oggetti (`'bank.n.04'`, *e.g.* un banco di pesci).

In [73]:
word = 'bank'
sentence = 'the bank can guarantee deposits will eventually cover future tuition costs because it invests in adjustable-rate mortgage securities'

print(simplified_lesk(word, sentence))

Synset('depository_financial_institution.n.01')


Si può constatare da questo primo esempio come già questa versione simplificata dell'algoritmo di Lesk, basata sul calcolo e il confronto della sovrapposizione delle *signature* dei vari sensi con il dato contesto, permetta una corretta disambiguazione del senso di una parola (almeno per un esempio immediato come quello in analisi).

#### Rimozione delle *stop words*

Analizzando gli overlap tra le varie *signature* e il dato contesto è possibile constatare come la maggiorparte delle sovrapposizioni siano composte da *stop words* (nel caso visto sopra, da *in* e *the*), parole non utili all'obiettivo di disambiguare il senso di una data parola.

Si decide quindi di implementare, utilizzando gli strumenti forniti da NLTK per il trattamento delle *stop words*, una versione semplificata dell'algoritmo di Lesk che ignori proprio quest'ultime, così da evitare che overlap composti in parte o nella totalità da *stop words* finiscano per oscurare (numericamente) overlap con parole utili al task di disambiguazione.

In [74]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def stopwordless_simplified_lesk(word, sentence):

  word_senses = wn.synsets(word)
  best_sense = word_senses[0]

  if len(word_senses) == 1: return best_sense

  max_overlap = 0

  # The context is the corresponding set of words contained in the given sentence (minus non-alphanumerical chars and stop words)
  context = set(remove_non_alphanumerical_chars(sentence).split(' ')).difference(stop_words)

  for word_sense in word_senses:

    # Compute current sense signature (definition + examples)
    signature = set(remove_non_alphanumerical_chars(word_sense.definition()).split(' '))
    for example in word_sense.examples():
      # Add current example words to the signature (minus non-alphanumerical chars)
      signature.update(remove_non_alphanumerical_chars(example).split(' '))
    signature = signature.difference(stop_words) # Remove stop words from signature

    overlap = len(signature.intersection(context)) # Compute the overlap between current signature and the given context

    if overlap > max_overlap:
      max_overlap = overlap
      best_sense = word_sense

  return best_sense

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lorenzo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Utilizzando l'esempio precedente sulla disambiguazione della parola *bank*, è possibile constatare come il risultato dell'algoritmo non vari, in quanto le parole in overlap per il senso (corretto) `'depository_financial_institution.n.01'` sono $\{deposits,\: mortgage,\: bank\}$, tre termini che non vengono filtrati non essendo *stop words*. D'ora in avanti l'implementazione dell'algoritmo presa in considerazione sarà questa.

In [75]:
print(stopwordless_simplified_lesk(word, sentence), stopwordless_simplified_lesk(word, sentence) == simplified_lesk(word, sentence))

Synset('depository_financial_institution.n.01') True


### Disambiguare frasi in SemCor

Come dal punto $1$ della consegna, si estraggono cinquanta frasi dal corpus SemCor e si va a disambiguare (almeno) un sostantivo per frase.

Per evitare disambiguazioni *banali*, si decide prima di selezionare i sostantivi *polisemici* dell'attuale frase (quelli con almeno due synset di tipo `NOUN` associati) per poi andarli a disambiguare utilizzando la versione semplificata dell'algoritmo di Lesk vista sopra.

In [76]:
def compute_ambiguous_nouns(word_list):
  words = set(word_list).difference(stop_words)
  ambiguous_nouns = []

  for word in words:
    word_senses = wn.synsets(word, pos=wn.NOUN)
    if word_senses and len(word_senses) > 1:
      ambiguous_nouns.append(word)

  return ambiguous_nouns

In [77]:
ambiguous_nouns = compute_ambiguous_nouns(sentence.split())

print(ambiguous_nouns)

for ambiguous_noun in ambiguous_nouns:
  print(f"   {ambiguous_noun} with {len(wn.synsets(ambiguous_noun, pos=wn.NOUN))} noun synsets")

['securities', 'future', 'guarantee', 'costs', 'bank', 'cover', 'tuition', 'deposits']
   securities with 9 noun synsets
   future with 3 noun synsets
   guarantee with 3 noun synsets
   costs with 4 noun synsets
   bank with 10 noun synsets
   cover with 10 noun synsets
   tuition with 2 noun synsets
   deposits with 9 noun synsets


Verificandone il funzionamento con la frase d'esempio (di sopra) è però evidente da subito una criticità: si vanno anche a considerare componenti della frase che in questo caso non hanno il ruolo di sostantivi, come *guarantee* e *cover*, che svolgono invece il ruolo di verbi.

È presente quindi, oltre a un livello di ambiguità semantica, anche un livello di ambiguità sintattica: sarà necessario utilizzare un PoS Tagger per filtrare i soli sostantivi o utilizzare un dataset già annotato con i corretti PoS (come nel caso di SemCor).

In [78]:
nltk.download('semcor')
from nltk.corpus import semcor

semcor_sentences = semcor.sents()
semcor_tagged_sentences = semcor.tagged_sents(tag='both')

[nltk_data] Downloading package semcor to /home/lorenzo/nltk_data...
[nltk_data]   Package semcor is already up-to-date!


#### SemCor

SemCor è un corpus annotato dove a ogni parola delle 37176 frasi presenti nel dataset è associato il corretto synset WordNet e il corretto tag PoS (Penn Treebank tagset) nel contesto della frase (l'associazione è stata fatta da umani, trattasi perciò di un *golden corpus*), è quindi possibile utilizzare SemCor per discriminare i sostantivi da scegliere per la disambiguazione (e, soprattutto, come benchmark per valutare le prestazioni della nostra implementazione).

Ogni termine in una frase (o insieme di termini, in quanto sono gestite anche le espressioni multi-termine) è rappresentato come un albero NLTK, con il suo tag PoS (l'etichetta dell'albero) e l'effettivo termine (o termini) come foglie.

In [79]:
def get_nouns_in_semcore_sentence(sent_id):

  noun_list = []

  for element in semcor_tagged_sentences[sent_id]:

      # This complex multi-part condition is needed (instead of a simpler
      # `if element.label() == 'NN'` from a SemCore PoS Tree) to cover those
      # rare cases when a noun hasn't a corresponding WordNet synset
      # (within our experience this happened just once with the noun *none*)
      if isinstance(element, nltk.Tree) and \
         isinstance(element.label(), nltk.corpus.reader.wordnet.Lemma) and \
         element[0].label() == 'NN':  # Fetch nouns only
            noun_list.append(" ".join(element.leaves()))

  return noun_list

Si utilizza la prima frase fornita da SemCor per valutare la bontà di quanto implementato finora.

In [80]:
print("Sentence:", " ".join(semcor_sentences[0]))

nouns = get_nouns_in_semcore_sentence(0)
print("Nouns:")
print("  ", nouns)

ambiguous_nouns = compute_ambiguous_nouns(nouns)
print("Ambiguous nouns:")
for ambiguous_noun in ambiguous_nouns:
  print(f"   {ambiguous_noun} with {len(wn.synsets(ambiguous_noun, pos=wn.NOUN))} noun synsets")

Sentence: The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place .
Nouns:
   ['Friday', 'investigation', 'Atlanta', 'primary election', 'evidence', 'irregularities']
Ambiguous nouns:
   evidence with 3 noun synsets
   Atlanta with 2 noun synsets
   investigation with 2 noun synsets
   irregularities with 4 noun synsets


#### Esecuzione su singolo batch

Si estraggono (le prime) cinquanta frasi dal corpus SemCor con sostantivi ambigui e di questi (per ogni frase) si va a disambiguare o il primo, o il *più ambiguo* (*i.e* quello con più synset di tipo `NOUN` associati) o il *meno ambiguo*. Una volta disambiguati si va a calcolare l'accuratezza confrontando il risultato con i valori contenuti in SemCor.

In [81]:
def compute_most_ambiguous_noun(noun_list):
  most_ambiguous_noun = None
  max_sense_n = 0

  for noun in noun_list:
    if len(wn.synsets(noun, pos=wn.NOUN)) > max_sense_n:
      max_sense_n = len(wn.synsets(noun, pos=wn.NOUN))
      most_ambiguous_noun = noun

  return most_ambiguous_noun

def compute_least_ambiguous_noun(noun_list):
  least_ambiguous_noun = noun_list[0]
  min_sense_n = len(wn.synsets(noun_list[0]))

  for noun in noun_list:
    if len(wn.synsets(noun, pos=wn.NOUN)) < min_sense_n:
      min_sense_n = len(wn.synsets(noun, pos=wn.NOUN))
      least_ambiguous_noun = noun

  return least_ambiguous_noun

In [82]:
# Get the corresponding synset of a given noun in a given sentence in the SemCor corpus
def get_semcor_synset(noun, sent_id):
  for element in semcor_tagged_sentences[sent_id]:
    if isinstance(element, nltk.Tree) and \
       isinstance(element.label(), nltk.corpus.reader.wordnet.Lemma) and \
       element[0].label() == 'NN' and \
       " ".join(element.leaves()) == noun:
          return element.label().synset()

# Get the first n SemCor sentences (id) which contain ambiguous nouns
def get_n_semcor_sentences_with_ambiguous_nouns(n):
  id_list = []
  i = 0

  while (n > 0):
    nouns = get_nouns_in_semcore_sentence(i)
    ambiguous_nouns = compute_ambiguous_nouns(nouns)
    if ambiguous_nouns:
      id_list.append(i)
      n = n - 1
    i = i + 1

  return id_list

In [83]:
sentences_number = 50

print("Fetching SemCor sentences with ambiguous nouns...")
sentences_ids = get_n_semcor_sentences_with_ambiguous_nouns(sentences_number)

print("Currently disambiguating...")

correct_disambiguations = 0

for sentence_id in sentences_ids:
  nouns = get_nouns_in_semcore_sentence(sentence_id)
  ambiguous_nouns = compute_ambiguous_nouns(nouns)

  # Set to compute_most_ambiguous_noun or to ambiguous_nouns[0] to change selection criterion
  noun_to_disambiguate = compute_least_ambiguous_noun(ambiguous_nouns)
  lesk_result = stopwordless_simplified_lesk(noun_to_disambiguate, " ".join(semcor_sentences[sentence_id]))
  semcor_annotation = get_semcor_synset(noun_to_disambiguate, sentence_id)

  if lesk_result == semcor_annotation:
    correct_disambiguations = correct_disambiguations + 1

print("Accuracy:", correct_disambiguations / len(sentences_ids))

Fetching SemCor sentences with ambiguous nouns...
Currently disambiguating...
Accuracy: 0.66


Con le prime cinquanta frasi con sostantivi ambigui l'accuratezza è la seguente:

- Meno ambiguo (*best case scenario*): $68\%$
- Più ambiguo (*worst case scenario*): $28\%$
- Primo sostantivo ambiguo (scelta aribitraria): $56\%$

#### Esecuzione su batch randomizzati


Come dal punto $2$ della consegna, si randomizza la selezione delle 50 frasi e la selezione del termine da disambiguare, e si va a restituire l'accuratezza media su (per esempio) 10 esecuzioni del programma.

In [84]:
import random

def get_random_ambiguous_noun(noun_list):
  return random.choice(noun_list)

# Get n random SemCor sentences (id) which contain ambiguous nouns (that haven't been already picked)
def get_n_random_semcor_sentences_with_ambiguous_nouns(n, previous_ids):
  id_list = []
  i = 0

  while (n > 0):

    # Generate a random id and check if it has already been picked
    random_id = random.randint(0, len(semcor_sentences)-1)

    while random_id in previous_ids:
      random_id = random.randint(0, len(semcor_sentences)-1)

    # Add a new random sentence id only if it hasn't already picked and if the sentence has ambiguous nouns
    nouns = get_nouns_in_semcore_sentence(random_id)
    ambiguous_nouns = compute_ambiguous_nouns(nouns)
    if ambiguous_nouns:
      id_list.append(random_id)
      n = n - 1

    # Append the current random id even if it's not useful in order to avoid future re-calculation
    previous_ids.append(random_id)

  return (id_list, previous_ids)

In [115]:
sentences_number = 50
iterations_number = 10

print(f"Currently disambiguating {iterations_number} batch of {sentences_number} sentences...")

cumulative_accuracy = 0

previous_ids = []

for i in range(0, iterations_number):
  (sentences_ids, previous_ids) = get_n_random_semcor_sentences_with_ambiguous_nouns(sentences_number, previous_ids)

  correct_disambiguations = 0

  for sentence_id in sentences_ids:
    nouns = get_nouns_in_semcore_sentence(sentence_id)
    ambiguous_nouns = compute_ambiguous_nouns(nouns)

    noun_to_disambiguate = get_random_ambiguous_noun(ambiguous_nouns) # Randomized ambiguous noun choice
    lesk_result = stopwordless_simplified_lesk(noun_to_disambiguate, " ".join(semcor_sentences[sentence_id]))
    semcor_annotation = get_semcor_synset(noun_to_disambiguate, sentence_id)

    if lesk_result == semcor_annotation:
      correct_disambiguations = correct_disambiguations + 1

  cumulative_accuracy = cumulative_accuracy + (correct_disambiguations / len(sentences_ids))

print("Average accuracy:", cumulative_accuracy / iterations_number)

Currently disambiguating 10 batch of 50 sentences...
Average accuracy: 0.476


Eseguendo più volte il codice è possibile constatare come l'accuratezza media oscilli tra il $40\%$ e il $50\%$, prestazioni al disopra del *worst case scenario* visto sopra ma comunque sufficientemente inferiori a quello migliore, e non migliori di un *random guessing*. Per ottenere prestazioni migliori è necessario implementare *Corpus Lesk*, così da poter guidare le scelte dei vari sensi utilizzando i più usati all'interno del corpus di SemCor.